# 🤖💬 Chatbot inteligente para geração de respostas em linguagem pt-br

Este projeto utiliza o modelo **PTT5 (Portuguese T5)**, uma adaptação do modelo **T5 (Text-to-Text Transfer Transformer)** para a língua portuguesa.

- 📐 **Arquitetura:** baseada no Transformer, com codificador e decodificador (encoder-decoder).
- 🧠 **Capacidade:** compreende instruções em linguagem natural e gera respostas coerentes e contextualizadas.
- 📚 **Aprendizado:** usa fine-tuning sobre dados em português via aprendizado por transferência.
- 🛠️ **Base:** modelo `unicamp-dl/ptt5-base-portuguese-vocab`.


> ⚠️ dataset.csv, requirements.txt incluídas no repositório

## 🧰 Parte 1: Ambiente e dependências

Nesta seção, definimos o ambiente necessário para executar o projeto, garantindo que todas as bibliotecas estejam corretamente instaladas e compatíveis com o modelo utilizado.

As etapas abaixo cobrem:

- Especificação da versão do Python e sistema operacional
- Organização do ambiente virtual (Conda recomendado)
- Instalação das dependências via `pip` ou `requirements.txt`
- Registro das bibliotecas utilizadas no desenvolvimento

> ⚠️ Ter um ambiente reprodutível é essencial para evitar conflitos de versão e garantir que o modelo funcione como esperado.



⚙️ 1.1: Ambiente e dependências utilizadas

O modelo foi treinado e executado em um ambiente com a seguinte configuração:

- 🐍 **Python**: 3.9.x (via Anaconda) 
- 💻 **Sistema Operacional**: Windows (compatível também com Linux)  
- 🧪 **Ambiente virtual**: criado com o **Conda**

📦 1.2: Bibliotecas essenciais

!pip install transformers==4.37.2
!pip install datasets==2.16.1
!pip install pandas==2.1.4
!pip install torch==2.1.0

🗃️ 1.3: Instalação das dependências

Caso queira utilizar todas as bibliotecas que foram utilizadas utilize o arquivo `requirements.txt`. Para instalar, ative o ambiente Conda desejado e execute:


```bash
pip install -r requirements.txt

---

## 🏋️‍♂️ Parte 2: Treinamento do modelo PTT5 com dados personalizados

Nesta seção, executamos todas as etapas relacionadas ao **processo de fine-tuning do modelo PTT5** em português, utilizando um dataset customizado.

As etapas abaixo cobrem:

- Importação de bibliotecas
- Carregamento e preparação do modelo
- Processamento e tokenização dos dados
- Definição de argumentos de treinamento
- Execução do treinamento com monitoramento de loss
- Salvamento do modelo final e uso para inferência


💻 Etapa 2.1: Importação das bibliotecas

        Importamos os módulos essenciais para manipular dados, preparar o modelo e realizar o treinamento com Transformers.



In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, TrainerCallback
from datasets import Dataset
import pandas as pd
import torch

📘 Etapa 2.2: Callback para monitorar a perda por época

        Esta função imprime a loss ao final de cada época durante o treinamento.


In [ ]:
class EpochLossPrinterCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"\n📘 [Época {int(state.epoch)}] Loss de Treinamento: {state.log_history[-1]['loss']:.4f}")


📥 Etapa 2.3: Carregando o modelo PTT5 e o tokenizer

        Utilizamos a versão base do modelo T5 treinado em português pela Unicamp.


In [ ]:
model_name = "unicamp-dl/ptt5-base-portuguese-vocab"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

📥 Etapa 2.4: Carregando e preparando o dataset CSV

        Unificamos as colunas do dataset em uma entrada textual formatada para o modelo.


In [ ]:
df = pd.read_csv("dataset.csv")
df["input_text"] = "tópico: " + df["topic"] + " | instrução: " + df["instruction"]
df["target_text"] = df["output"]

📚 Etapa 2.5: Transformação do DataFrame em um objeto Dataset 

        Transformamos o DataFrame em um objeto Dataset para integração com o `Trainer`.

In [ ]:
dataset = Dataset.from_pandas(df[["input_text", "target_text"]])

🧩 Etapa 2.6: Tokenização dos dados

        Aplicamos truncamento, padding e preparamos os pares entrada/saída para o treinamento.

In [ ]:
def tokenize(batch):
    input_enc = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=128)
    target_enc = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=128)
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

tokenized_dataset = dataset.map(tokenize, batched=True)

📘 Etapa 2.7: Configuração dos hiperparâmetros de treinamento

        Definimos estratégia de salvamento, taxa de aprendizado, uso de FP16 e número de épocas.


In [ ]:
training_args = TrainingArguments(
    output_dir="./ptt5-modelo-treinado",       # Pasta onde os checkpoints do modelo serão salvos
    overwrite_output_dir=True,                 # Sobrescreve a pasta de saída se ela já existir
    per_device_train_batch_size=30,            # Tamanho do batch por dispositivo (ex: GPU) durante o treinamento
    gradient_accumulation_steps=2,             # Acumula gradientes por 2 batches antes de atualizar os pesos (simula batch_size=60)
    learning_rate=3e-4,                        # Taxa de aprendizado inicial (valor relativamente alto, ideal para fine-tuning)
    num_train_epochs=30,                       # Número total de épocas de treinamento (passadas completas pelo dataset)
    save_strategy="epoch",                     # Salva um checkpoint do modelo ao final de cada época
    weight_decay=0.005,                        # Taxa de decaimento dos pesos (regularização para evitar overfitting)
    warmup_steps=100,                          # Número de steps com aprendizado mais suave no início (warm-up)
    logging_steps=10,                          # Frequência (em steps) com que as métricas de treino serão logadas no console
    fp16=torch.cuda.is_available(),            # Ativa treinamento em precisão mista (FP16) se houver GPU compatível (mais rápido e leve)
    report_to="none",                          # Desativa integração com sistemas de logging externos (ex: TensorBoard, WandB)
    save_total_limit=3                         # Mantém no máximo 3 checkpoints salvos no disco (os mais recentes)
)


⚙️ Etapa 2.8: Inicialização do Trainer

        Instanciamos o `Trainer` com o modelo, os dados tokenizados e o callback para logging da loss.


In [ ]:
trainer = Trainer(
    model=model,                               # Modelo PTT5 carregado e pronto para ser treinado
    args=training_args,                        # Conjunto de argumentos de treinamento definidos anteriormente (TrainingArguments)
    train_dataset=tokenized_dataset,           # Dataset já tokenizado que será usado para o treinamento
    callbacks=[EpochLossPrinterCallback()]     # Lista de callbacks personalizados; neste caso, imprime a loss ao final de cada época
)


🚀 Etapa 2.9: Início do treinamento

        Executamos o fine-tuning do modelo com os dados fornecidos.


In [ ]:
trainer.train()

💾 Etapa 2.10: Salvando o modelo e o tokenizer treinados

        Após o treinamento, salvamos o modelo fine-tunado e seu tokenizer para uso posterior em inferência.


In [ ]:
model.save_pretrained("./ptt5-modelo-final")
tokenizer.save_pretrained("./ptt5-modelo-final")

---

## 💬 Parte 3: Uso do modelo treinado para geração de respostas

Com o modelo PTT5 já treinado e salvo, esta seção aborda as etapas necessárias para utilizá-lo em tempo de execução.

Aqui, veremos:

- Como **carregar o modelo salvo** e seu tokenizer
- Definir uma função `responder()` que gere respostas a partir de perguntas em linguagem natural
- Realizar **testes práticos com perguntas reais** simulando um chatbot
- Explorar diferentes configurações de geração, como `temperature`, `top_k` e `top_p`

> 📌 Esta seção é fundamental para validar a performance prática do modelo após o fine-tuning.


🔁 Etapa 3.1: Carregando o modelo final para uso

        Agora vamos utilizar o modelo PTT5 fine-tunado para responder perguntas reais em português.

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Caminho para o modelo salvo
model_dir = "./ptt5-modelo-final"

# Carregando o tokenizer e o modelo
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

📘 Etapa 3.2: Função para geração de respostas

        Criamos uma função `responder()` que recebe uma instrução e retorna uma resposta gerada pelo modelo.


In [ ]:
def responder(pergunta):
    # Concatena o prefixo "instrução: " com a pergunta do usuário (formato usado no treinamento do modelo)
    entrada = "instrução: " + pergunta

    # Tokeniza a entrada textual para o formato aceito pelo modelo
    inputs = tokenizer(
        entrada,                 # Texto de entrada para o modelo
        return_tensors="pt",     # Retorna tensores do PyTorch
        truncation=True,         # Trunca o texto se ultrapassar o limite máximo
        padding=True,            # Aplica padding para atingir o tamanho fixo
        max_length=128           # Define o tamanho máximo da sequência de entrada
    )

    # Gera a saída textual usando o modelo treinado
    outputs = model.generate(
        **inputs,                # Passa os tensores tokenizados como entrada
        max_length=64,           # Limita o tamanho da resposta gerada
        do_sample=True,          # Ativa amostragem aleatória (mais criativo)
        top_k=50,                # Considera apenas os 50 tokens mais prováveis (Top-K sampling)
        top_p=0.95,              # Aplica nucleus sampling (Top-P), acumulando até 95% de probabilidade
        temperature=0.7,         # Controla a aleatoriedade (quanto menor, mais conservador)
        repetition_penalty=1.2,  # Penaliza repetições para evitar respostas redundantes
        num_return_sequences=1   # Gera apenas uma resposta
    )

    # Decodifica a resposta gerada de volta para string legível (removendo tokens especiais)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


💬 Etapa 3.3: Fazendo perguntas ao modelo treinado
        
        Abaixo, alguns exemplos reais testando o modelo com instruções típicas.

In [ ]:
print(responder("A máquina aceita cartão?"))
print(responder("Tem coca-cola?"))
print(responder("Se um turista usar a máquina, ele vai conseguir entender?"))
print(responder("Posso pagar com boleto?"))
print(responder("Quais são a forma de pagamento?"))
print(responder("Tem bebida com gás?"))
print(responder("Tem opção de refrigerante sem cafeína?"))
print(responder("O refrigerante não sai da máquina, como devo proceder?"))
print(responder("Qual telefone de suporte?"))
print(responder("Você tem laranja?"))


---

## 🗂️ Parte 4: Construção do dataset

Nesta seção, explicamos como foi construído o dataset utilizado no fine-tuning do modelo PTT5, focado na geração de respostas em português com base em instruções específicas.

As etapas abaixo cobrem:

- Estrutura das colunas (`topic`, `instruction`, `output`)
- Estratégia de formulação das instruções e respostas
- Geração e organização dos exemplos em CSV
- Objetivo dos dados: simular interações reais com um chatbot

> 🧠 A qualidade e variedade do dataset são fatores decisivos para o desempenho do modelo. Neste projeto, priorizamos instruções curtas, diretas e contextualizadas, refletindo situações reais de atendimento automatizado.


🎯 4.1: Estratégia de construção dos exemplos

> A partir de 5 perguntas humanas por tópico, você poderá gerar as 100 por tópico

Para garantir a robustez e a naturalidade das respostas geradas pelo modelo, cada tópico do dataset foi estruturado com 100 exemplos balanceados em quatro categorias distintas:

- **30 perguntas variadas com estrutura direta**, explorando diferentes formas de expressar a mesma intenção;
- **20 perguntas com variações de estilo e contexto**, incluindo registros formais, informais e construções regionais;
- **30 casos de erro, exceção ou negativa**, representando limitações reais do sistema (ex: produto indisponível, recurso inexistente);
- **20 perguntas indiretas, curiosidades ou detalhes contextuais**, que exigem inferência semântica ou compreensão mais sutil.

O conjunto total abrange **11 tópicos específicos** relacionados ao funcionamento da máquina de venda: `sabores`, `acessibilidade`, `horário de funcionamento`, `idioma`, `promoções`, `contato`, `reembolso`, `falhas`, `uso da máquina`, `produtos` e `pagamento`.

Essa abordagem visa simular interações autênticas com usuários em cenários reais, aumentando a capacidade do modelo de generalizar, recusar adequadamente e manter coerência mesmo diante de instruções incomuns ou incompletas.


📌 4.1.1 Estratégia para gerar 30 perguntas variadas com estrutura diferente

🎯 Objetivo: Gerar variações de forma (estrutura gramatical) mantendo a mesma intenção da pergunta.

📦 Técnicas utilizadas:
- Parafraseamento com modelos do Hugging Face
- Geração de variações com backtranslation
- Substituição por sinônimos com spaCy ou NLPAug

🔧 Bibliotecas:
- transformers
- sentence-transformers
- spaCy
- nlpaug

`Exemplo usando Hugging Face para parafrasear com um modelo pré-treinado`

In [ ]:
from transformers import pipeline

paraphraser = pipeline("text2text-generation", model="Vamsi/T5-paraphraser")

pergunta_original = "Quais são os sabores disponíveis?"
variacoes = paraphraser(f"paraphrase: {pergunta_original} </s>", max_length=64, num_return_sequences=5, do_sample=True)

for v in variacoes:
    print("-", v['generated_text'])

✍️ 4.1.2 Estratégia para gerar 20 perguntas com variações de estilo e contexto

🎯 Objetivo: Criar versões formais, informais e regionais da mesma pergunta.

📦 Técnicas utilizadas:
- Substituição lexical com dicionários (gírias, formalismos)
- Reescrita com regras baseadas em regex ou spaCy
- Augmenters com estilo (ex: informal/texting)

🔧 Bibliotecas:
- nlpaug (contextualWordEmbs + synonym)
- pandas + regex + substituições manuais

`Exemplo simples: troca informal-formal via regras`

In [ ]:
pergunta = "Cês têm suco de uva?"

substituicoes = {
    "cês": "vocês",
    "tem": "têm",
    "suco": "bebida",
}

for g in substituicoes:
    pergunta = pergunta.replace(g, substituicoes[g])

print("Formal:", pergunta)

❌ 4.1.3 Estratégia para gerar 30 exemplos de erro, exceção ou negativa

🎯 Objetivo: Criar perguntas sobre limitações do sistema e situações em que o chatbot deve negar ou informar falha.

📦 Estratégias:
- Adição de elementos de falha: "e se não funcionar?", "e se acabar o produto?"
- Combinação com palavras-chave negativas: "não", "acabou", "quebrado"
- Geração por padrão com templates de exceção

🔧 Técnicas:
- Template-based generation com variações programáticas
- Controle por palavras-chave

`Gerador simples de perguntas negativas`

In [ ]:

produto = "limonada"
templates = [
    f"E se acabar a {produto}?",
    f"O que acontece se a máquina travar na hora da {produto}?",
    f"A máquina avisa quando não tem mais {produto}?",
    f"A máquina pode errar o sabor da {produto}?",
    f"Posso pedir reembolso se a {produto} não sair?",
]

for p in templates:
    print("-", p)

💡 4.1.4 Estratégia para gerar 20 perguntas indiretas, curiosas ou contextuais

🎯 Objetivo: Criar perguntas que não pedem algo diretamente, mas fazem referência contextual, emocional ou hipotética.

📦 Estratégias:
- Geração com prompts do tipo "E se...?", "Será que...?", "Qual é o melhor para..."
- Uso de sentenças interrogativas abertas
- Inspiração em FAQs, experiências de usuários reais

🔧 Técnicas:
- Manual com apoio de GPT ou modelo instruído
- Criação de padrões com linguagem subjetiva

🔧 Bibliotecas:
- transformers (para gerar exemplos a partir de poucos prompts)

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", max_length=60)

prompt = "E se eu esquecer a carteira na hora de pagar?"
resposta = generator(prompt, do_sample=True, top_p=0.9, temperature=0.7, num_return_sequences=3)

for r in resposta:
    print("-", r["generated_text"])